<b>Data mining project - 2020/21</b><br>
<b>Author</b>: [Alexandra Bradan](https://github.com/alexandrabradan)<br>
<b>Python version</b>: 3.x<br>
<b>Last update: 20/11/2020<b>

In [58]:
%matplotlib inline

# general libraries
import sys
import math
import operator
import itertools
import pydotplus
import collections
import missingno as msno
from pylab import MaxNLocator
from collections import Counter
from collections import defaultdict
from IPython.display import Image

# pandas libraries
import pandas as pd
from pandas import DataFrame
from pandas.testing import assert_frame_equal

# visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

# numpy libraries
import numpy as np
from numpy import std
from numpy import mean
from numpy import arange
from numpy import argmax
from numpy import unique
from numpy import percentile

# scipy libraries
import scipy.stats as stats
from scipy.stats import kstest
from scipy.stats import normaltest

# sklearn libraries
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer  # explicitly require this experimental feature
from sklearn.impute import IterativeImputer

from sklearn import tree
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.pipeline import make_pipeline as imbmake_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, recall_score, precision_score, classification_report, roc_auc_score 

In [59]:
data_directory = "../../../data/"
plot_directory = "../../../plots/DataUnderstanding/"
TR_cleaned_file = data_directory + "Discretized_Train_HR_Employee_Attrition.csv"
TS_file = data_directory + "Discretized_Test_HR_Employee_Attrition.csv"

In [60]:
df_cleaned = pd.read_csv(TR_cleaned_file, sep=",") 
df_ts = pd.read_csv(TS_file, sep=",") 

In [61]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Age                                     236 non-null    int64  
 1   Attrition                               236 non-null    int64  
 2   BusinessTravel_Non-Travel               236 non-null    int64  
 3   BusinessTravel_Travel_Rarely            236 non-null    int64  
 4   BusinessTravel_Travel_Frequently        236 non-null    int64  
 5   DistanceFromHome                        236 non-null    int64  
 6   Education                               236 non-null    int64  
 7   EnvironmentSatisfaction                 236 non-null    int64  
 8   Gender                                  236 non-null    int64  
 9   JobInvolvement                          236 non-null    int64  
 10  JobLevel                                236 non-null    int64 

<h2> Define Training set </h2>

<h6> Removing OverallSatisfaction not used for discretisation and DT classification</h6>

In [62]:
del df_cleaned["OverallSatisfaction"]

In [63]:
y_train = df_cleaned['Attrition']
df1 = df_cleaned.copy()
X_train = df1.drop('Attrition', axis=1)
print(X_train.shape)
print(y_train.shape)

(883, 32)
(883,)


<h2> Define Test set </h2>

<h6> Removing Age, MonthlyHours, OverallSatisfaction not used for discretisation and DT classification</h6>

In [64]:
to_drop = ["Age", "MonthlyHours", "OverallSatisfaction", "JobRole_Manager"]

# drop features 
for column_name in to_drop:
    del df_ts[column_name]

# check dropping output
print(df_ts.shape)

(236, 33)


In [66]:
y_test = df_cleaned['Attrition']
df2 = df_ts.copy()
X_test = df2.drop('Attrition', axis=1)
print(X_test.shape)
print(y_test.shape)

(236, 32)
(883,)


<h2>Predict on Test set</h2>